# Cross-subject decoding rule representations

#### Takuya Ito
#### 2/28/2018
##### Last updated: 11/20/2019

In [4]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import multiprocessing as mp
import scipy.stats as stats
import os
os.environ['OMP_NUM_THREADS'] = str(1)
import statsmodels.sandbox.stats.multicomp as mc
import seaborn as sns
import h5py
import tools_group_rsa as tools_group
import nibabel as nib
tools_group = reload(tools_group)

sns.set_style("whitegrid")
plt.rcParams["font.family"] = "FreeSans"


In [5]:
# Excluding 084
subjNums = ['013','014','016','017','018','021','023','024','026','027','028','030','031','032','033',
            '034','035','037','038','039','040','041','042','043','045','046','047','048','049','050',
            '053','055','056','057','058','062','063','066','067','068','069','070','072','074','075',
            '076','077','081','085','086','087','088','090','092','093','094','095','097','098','099',
            '101','102','103','104','105','106','108','109','110','111','112','114','115','117','119',
            '120','121','122','123','124','125','126','127','128','129','130','131','132','134','135',
            '136','137','138','139','140','141']



basedir = '/projects3/SRActFlow/'

# Using final partition
networkdef = np.loadtxt('/projects3/NetworkDiversity/data/network_partition.txt')
networkorder = np.asarray(sorted(range(len(networkdef)), key=lambda k: networkdef[k]))
networkorder.shape = (len(networkorder),1)
# network mappings for final partition set
networkmappings = {'fpn':7, 'vis1':1, 'vis2':2, 'smn':3, 'aud':8, 'lan':6, 'dan':5, 'con':4, 'dmn':9, 
                   'pmulti':10, 'none1':11, 'none2':12}
networks = networkmappings.keys()

xticks = {}
reorderednetworkaffil = networkdef[networkorder]
for net in networks:
    netNum = networkmappings[net]
    netind = np.where(reorderednetworkaffil==netNum)[0]
    tick = np.max(netind)
    xticks[tick] = net

## General parameters/variables
nParcels = 360
nSubjs = len(subjNums)

glasserfile2 = '/projects/AnalysisTools/ParcelsGlasser2016/Q1-Q6_RelatedParcellation210.LR.CorticalAreas_dil_Colors.32k_fs_RL.dlabel.nii'
glasser2 = nib.load(glasserfile2).get_data()
glasser2 = np.squeeze(glasser2)

sortednets = np.sort(xticks.keys())
orderednetworks = []
for net in sortednets: orderednetworks.append(xticks[net])
    
networkpalette = ['royalblue','slateblue','paleturquoise','darkorchid','limegreen',
                  'lightseagreen','yellow','orchid','r','peru','orange','olivedrab']
networkpalette = np.asarray(networkpalette)

OrderedNetworks = ['VIS1','VIS2','SMN','CON','DAN','LAN','FPN','AUD','DMN','PMM','VMM','ORA']

## 1.0 Load data for LOGIC rules

In [9]:
# gsr = True
nStims = 4
data_task = np.zeros((len(glasser2),nStims,len(subjNums)))
rule = 'Logic'

scount = 0
for subj in subjNums:
    data_task[:,:,scount] = tools_group.loadInputActivity(subj,rule)
    scount += 1

/home/tito/miniconda2/lib/python2.7/site-packages/ipykernel_launcher.py:8: ComplexWarning: Casting complex values to real discards the imaginary part
  


#### Run decoding 

In [10]:
nproc = 30
ncvs = 1
rois = np.arange(nParcels)

distances_baseline_logic = tools_group.conditionDecodings(data_task, rois, motorOutput=False, ncvs=ncvs, nproc=nproc)

#### Compute statistics

In [11]:
rule = 'Logic'
statistics_logic = np.zeros((len(rois),3)) # acc, q, acc_thresh
for roicount in range(len(rois)):
    ntrials = distances_baseline_logic.shape[1]
    p = stats.binom_test(np.mean(distances_baseline_logic[roicount,:])*ntrials,n=ntrials,p=1/float(nStims))
    if np.mean(distances_baseline_logic[roicount,:])>1/float(nStims):
        p = p/2.0
    else:
        p = 1.0-p/2.0
        

    statistics_logic[roicount,0] = np.mean(distances_baseline_logic[roicount,:])
    statistics_logic[roicount,1] = p

h0, qs = mc.fdrcorrection0(statistics_logic[:,1])
for roicount in range(len(rois)):
    statistics_logic[roicount,1] = qs[roicount]
    statistics_logic[roicount,2] = h0[roicount]*statistics_logic[roicount,0]
    
# Count number of significant ROIs for LH decoding
sig_ind = np.where(statistics_logic[:,1]<0.05)[0]
print 'Number of ROIs significant for', rule, 'Stimuli:', sig_ind.shape[0]
print 'Accuracies:', statistics_logic[sig_ind,0]

#### Map back to surface
# Put all data into a single matrix (since we only run a single classification)
inputStim = np.zeros((glasser2.shape[0],3))

roicount = 0
for roi in rois:
    vertex_ind = np.where(glasser2==roi+1)[0]
    inputStim[vertex_ind,0] = statistics_logic[roicount,0]
    inputStim[vertex_ind,1] = statistics_logic[roicount,1]
    inputStim[vertex_ind,2] = statistics_logic[roicount,2]

    roicount += 1

np.savetxt('/projects3/SRActFlow/data/results/GroupfMRI/RuleDecoding/LogicRule_Regions.csv', np.where(statistics_logic[:,1]<0.05)[0], delimiter=',')
    
#### 
# Write file to csv and run wb_command
outdir = '/projects3/SRActFlow/data/results/GroupfMRI/RuleDecoding/'
filename = rule + 'Decoding'
np.savetxt(outdir + filename + '.csv', inputStim,fmt='%s')
wb_file = filename + '.dscalar.nii'
wb_command = 'wb_command -cifti-convert -from-text ' + outdir + filename + '.csv ' + glasserfile2 + ' ' + outdir + wb_file + ' -reset-scalars'
os.system(wb_command)


Number of ROIs significant for Logic Stimuli: 251
Accuracies: [0.3671875  0.3046875  0.36197917 0.36197917 0.38541667 0.296875
 0.375      0.34635417 0.36979167 0.359375   0.35416667 0.359375
 0.375      0.34895833 0.35416667 0.31510417 0.3359375  0.3046875
 0.3125     0.40885417 0.33333333 0.359375   0.35677083 0.36197917
 0.3515625  0.33072917 0.29427083 0.46354167 0.43489583 0.36197917
 0.32552083 0.32291667 0.3515625  0.34114583 0.3203125  0.296875
 0.30208333 0.40364583 0.29166667 0.31510417 0.34635417 0.38802083
 0.328125   0.40625    0.38020833 0.29166667 0.34375    0.39322917
 0.3046875  0.42447917 0.296875   0.30989583 0.35677083 0.32552083
 0.34895833 0.44010417 0.38020833 0.41927083 0.30729167 0.32552083
 0.30729167 0.40364583 0.35677083 0.32291667 0.29947917 0.35416667
 0.29166667 0.36197917 0.30208333 0.3203125  0.3515625  0.36979167
 0.30208333 0.29166667 0.359375   0.34375    0.30729167 0.36979167
 0.30208333 0.41666667 0.36197917 0.40364583 0.328125   0.29427083
 0.3567

0

## 2.0 Load data for SENSORY rules

In [13]:
# gsr = True
nStims = 4
data_task = np.zeros((len(glasser2),nStims,len(subjNums)))
rule = 'Sensory'

scount = 0
for subj in subjNums:
    data_task[:,:,scount] = tools_group.loadInputActivity(subj,rule)
    scount += 1

/home/tito/miniconda2/lib/python2.7/site-packages/ipykernel_launcher.py:8: ComplexWarning: Casting complex values to real discards the imaginary part
  


#### Run decoding 

In [14]:
nproc = 30
ncvs = 1
rois = np.arange(nParcels)

distances_baseline_sensory = tools_group.conditionDecodings(data_task, rois, motorOutput=False, ncvs=ncvs, nproc=nproc)

#### Compute statistics

In [15]:
rule = 'Sensory'
statistics_sensory = np.zeros((len(rois),3)) # acc, q, acc_thresh
for roicount in range(len(rois)):
    ntrials = distances_baseline_sensory.shape[1]
    p = stats.binom_test(np.mean(distances_baseline_sensory[roicount,:])*ntrials,n=ntrials,p=1/float(nStims))
    if np.mean(distances_baseline_sensory[roicount,:])>1/float(nStims):
        p = p/2.0
    else:
        p = 1.0-p/2.0
        

    statistics_sensory[roicount,0] = np.mean(distances_baseline_sensory[roicount,:])
    statistics_sensory[roicount,1] = p

h0, qs = mc.fdrcorrection0(statistics_sensory[:,1])
for roicount in range(len(rois)):
    statistics_sensory[roicount,1] = qs[roicount]
    statistics_sensory[roicount,2] = h0[roicount]*statistics_sensory[roicount,0]
    
# Count number of significant ROIs for LH decoding
sig_ind = np.where(statistics_sensory[:,1]<0.05)[0]
print 'Number of ROIs significant for', rule, 'Stimuli:', sig_ind.shape[0]
print 'Accuracies:', statistics_sensory[sig_ind,0]

#### Map back to surface
# Put all data into a single matrix (since we only run a single classification)
inputStim = np.zeros((glasser2.shape[0],3))

roicount = 0
for roi in rois:
    vertex_ind = np.where(glasser2==roi+1)[0]
    inputStim[vertex_ind,0] = statistics_sensory[roicount,0]
    inputStim[vertex_ind,1] = statistics_sensory[roicount,1]
    inputStim[vertex_ind,2] = statistics_sensory[roicount,2]

    roicount += 1

np.savetxt('/projects3/SRActFlow/data/results/GroupfMRI/RuleDecoding/SensoryRule_Regions.csv', np.where(statistics_sensory[:,1]<0.05)[0], delimiter=',')
    
#### 
# Write file to csv and run wb_command
outdir = '/projects3/SRActFlow/data/results/GroupfMRI/RuleDecoding/'
filename = rule + 'Decoding'
np.savetxt(outdir + filename + '.csv', inputStim,fmt='%s')
wb_file = filename + '.dscalar.nii'
wb_command = 'wb_command -cifti-convert -from-text ' + outdir + filename + '.csv ' + glasserfile2 + ' ' + outdir + wb_file + ' -reset-scalars'
os.system(wb_command)


Number of ROIs significant for Sensory Stimuli: 84
Accuracies: [0.41927083 0.38541667 0.32552083 0.390625   0.33072917 0.3515625
 0.34375    0.32552083 0.36979167 0.32291667 0.31770833 0.31770833
 0.3515625  0.38020833 0.3046875  0.30989583 0.31510417 0.3203125
 0.33072917 0.35677083 0.31510417 0.3046875  0.31770833 0.31510417
 0.30989583 0.34375    0.3203125  0.328125   0.3984375  0.33072917
 0.32552083 0.3203125  0.34635417 0.36458333 0.32552083 0.32552083
 0.39322917 0.4140625  0.34114583 0.31770833 0.34375    0.37239583
 0.36979167 0.31770833 0.33072917 0.30208333 0.3125     0.34375
 0.30729167 0.4296875  0.40104167 0.37760417 0.37239583 0.328125
 0.31510417 0.32291667 0.31770833 0.33854167 0.3046875  0.34895833
 0.3359375  0.3203125  0.3046875  0.38020833 0.30208333 0.3046875
 0.3125     0.30208333 0.34375    0.30729167 0.328125   0.31770833
 0.30989583 0.31510417 0.3046875  0.34114583 0.3125     0.3125
 0.32552083 0.34895833 0.32291667 0.34895833 0.32291667 0.30208333]


0

## 3.0 Load data for MOTOR rules

In [17]:
# gsr = True
nStims = 4
data_task = np.zeros((len(glasser2),nStims,len(subjNums)))
rule = 'Motor'

scount = 0
for subj in subjNums:
    data_task[:,:,scount] = tools_group.loadInputActivity(subj,rule)
    scount += 1

/home/tito/miniconda2/lib/python2.7/site-packages/ipykernel_launcher.py:8: ComplexWarning: Casting complex values to real discards the imaginary part
  


#### Run decoding 

In [18]:
nproc = 30
ncvs = 1
rois = np.arange(nParcels)

distances_baseline_motor = tools_group.conditionDecodings(data_task, rois, motorOutput=False, ncvs=ncvs, nproc=nproc)

#### Compute statistics

In [19]:
rule = 'Motor'
statistics_motor = np.zeros((len(rois),3)) # acc, q, acc_thresh
for roicount in range(len(rois)):
    ntrials = distances_baseline_motor.shape[1]
    p = stats.binom_test(np.mean(distances_baseline_motor[roicount,:])*ntrials,n=ntrials,p=1/float(nStims))
    if np.mean(distances_baseline_motor[roicount,:])>1/float(nStims):
        p = p/2.0
    else:
        p = 1.0-p/2.0
        

    statistics_motor[roicount,0] = np.mean(distances_baseline_motor[roicount,:])
    statistics_motor[roicount,1] = p

h0, qs = mc.fdrcorrection0(statistics_motor[:,1])
for roicount in range(len(rois)):
    statistics_motor[roicount,1] = qs[roicount]
    statistics_motor[roicount,2] = h0[roicount]*statistics_motor[roicount,0]
    
# Count number of significant ROIs for LH decoding
sig_ind = np.where(statistics_motor[:,1]<0.05)[0]
print 'Number of ROIs significant for', rule, 'Stimuli:', sig_ind.shape[0]
print 'Accuracies:', statistics_motor[sig_ind,0]

#### Map back to surface
# Put all data into a single matrix (since we only run a single classification)
inputStim = np.zeros((glasser2.shape[0],3))

roicount = 0
for roi in rois:
    vertex_ind = np.where(glasser2==roi+1)[0]
    inputStim[vertex_ind,0] = statistics_motor[roicount,0]
    inputStim[vertex_ind,1] = statistics_motor[roicount,1]
    inputStim[vertex_ind,2] = statistics_motor[roicount,2]

    roicount += 1

np.savetxt('/projects3/SRActFlow/data/results/GroupfMRI/RuleDecoding/MotorRule_Regions.csv', np.where(statistics_motor[:,1]<0.05)[0], delimiter=',')
    
#### 
# Write file to csv and run wb_command
outdir = '/projects3/SRActFlow/data/results/GroupfMRI/RuleDecoding/'
filename = rule + 'Decoding'
np.savetxt(outdir + filename + '.csv', inputStim,fmt='%s')
wb_file = filename + '.dscalar.nii'
wb_command = 'wb_command -cifti-convert -from-text ' + outdir + filename + '.csv ' + glasserfile2 + ' ' + outdir + wb_file + ' -reset-scalars'
os.system(wb_command)


Number of ROIs significant for Motor Stimuli: 105
Accuracies: [0.36979167 0.3125     0.3359375  0.390625   0.35416667 0.49479167
 0.5        0.390625   0.3515625  0.30989583 0.3046875  0.30729167
 0.40104167 0.33072917 0.32552083 0.41927083 0.35416667 0.359375
 0.3828125  0.38541667 0.4921875  0.43489583 0.484375   0.4140625
 0.328125   0.328125   0.32552083 0.31770833 0.34375    0.32291667
 0.30208333 0.30208333 0.484375   0.31510417 0.34375    0.40625
 0.34895833 0.29947917 0.31510417 0.359375   0.34375    0.30208333
 0.3125     0.3125     0.29947917 0.375      0.44010417 0.3046875
 0.31510417 0.30989583 0.32291667 0.3203125  0.36979167 0.33333333
 0.3671875  0.29947917 0.30208333 0.3984375  0.36197917 0.32552083
 0.3046875  0.5        0.51041667 0.33854167 0.3125     0.3046875
 0.3203125  0.328125   0.4296875  0.40625    0.40364583 0.43229167
 0.33072917 0.3046875  0.34895833 0.34375    0.31510417 0.31510417
 0.4921875  0.4609375  0.47135417 0.41666667 0.390625   0.33333333
 0.39843

0

# Evaluate Overlap of accuracy/representations 

In [20]:
rois = np.arange(nParcels)
nStims = 4

statistics_all = np.multiply(statistics_logic[:,1]<0.05,statistics_sensory[:,1]<0.05)
statistics_all = np.multiply(statistics_all,statistics_motor[:,1]<0.05)

print 'Number of signifiicant overlaps:', np.sum(statistics_all)
        
decodingROI = np.zeros((glasser2.shape[0],4)) # unthresh acc, thresh acc
sig_acc = np.multiply(statistics_motor[:,0],statistics_motor[:,1]<0.05)
for roi in range(nParcels):
    roi_ind = np.where(glasser2==roi+1)[0]

    decodingROI[roi_ind,0] = statistics_all[roi]
    decodingROI[roi_ind,1] = np.multiply(statistics_all[roi],statistics_logic[roi,0])
    decodingROI[roi_ind,2] = np.multiply(statistics_all[roi],statistics_sensory[roi,0])
    decodingROI[roi_ind,3] = np.multiply(statistics_all[roi],statistics_motor[roi,0])

# Compute effect size baseline (information content)
outdir = '/projects3/SRActFlow/data/results/GroupfMRI/RuleDecoding/'
filename = 'OverlapRuleDecoding_64k'
np.savetxt(outdir + filename + '.csv', decodingROI,fmt='%s')
wb_command = 'wb_command -cifti-convert -from-text ' + outdir + filename + '.csv ' + glasserfile2 + ' ' + outdir + filename + '.dscalar.nii -reset-scalars'
os.system(wb_command)


Number of signifiicant overlaps: 29


0